# Upload non-spatial table to the sandbox
### Match format to existing table in  main portion of database

In [1]:
import sys
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
sys.path.append("C:/Users/ehbaker/Documents/Python/Repos/ice2O") #Path to where DBImport.py is saved
import DbImport #User defined module in the folder added to path in line above.
import numpy as np
import settings

**User supplied criteria**

In [2]:
#For draw wire
db_table='draw_wire' #name of table in the database which you want to copy
pth=(r"Q:\Project Data\GlacierData\WOLVERINE\Draw_Wire\data\processed\draw_wire_database.csv") #path to csv for upload

Connect to database:

In [3]:
cs=settings.import_cs() #user-defined module to store connection info
engine = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + cs['dbname'])

Read in csv for upload:

In [11]:
#Read in csv for upload
df=pd.read_csv(pth) #read to dataframe
#Extract format from existing table in main database
types=DbImport.define_db_table_format(df, db_table, engine)

Column names and Posgres data types for existing database table:

In [12]:
types

,attname,type
0,station_name,text
1,extension,double precision
2,datetime,timestamp without time zone
3,gid,integer


Check for matching column names and order

In [13]:
#Check to see if the columns in the new data frame match the ones in the database (order not important)
columns_match=set(list(types['attname'])) ==set(list(df))
columns_match

True

In [14]:
#Reorder columns to match order of database table (important only for display)
df=df[list(types['attname'])].copy() #the copy is neccesary to overwrite the initial dataframe with re-ordered values

Set primary key in new table to match that in databse:

In [16]:
#If numeric, add IDs sequentially, starting +1 from the current max
res=DbImport.pkey_NameAndType(db_table, engine)
pkey=res['attname'][0]
pkey_type=res['data_type'][0]
if pkey_type in ['smallint', 'integer', 'bigint', 'decimal', 'numeric', 'real', 'double precision', 'smallserial', 'serial', 'bigserial']:
    print ("Primary Key = Numeric \nAdding the primary key and unique IDs to rows of table being appended")
    df=DbImport.add_sequential_IDs_to_pkey(df, db_table, engine) #overwrite table
else:
    print("Primary ID is not Numeric; must be updated manually")

Primary key for draw_wire is: gid
Primary Key = Numeric 
Adding the primary key and unique IDs to rows of table being appended
Primary key for draw_wire is: gid


In [9]:
df[0:3] #look at resulting table, with updated primary ID

,station_name,extension,datetime,gid
0,Wolverine_siteEC,0.000000,5/7/2016 16:05,68097
1,Wolverine_siteEC,-0.010261,5/7/2016 17:05,68098
2,Wolverine_siteEC,-0.020799,5/7/2016 18:05,68099


Import table to database

In [17]:
#Connect to sandbox
engine_sand = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + 'sandbox')

In [18]:
#Add subset of new table to sandbox
df=df.sample(30) #overwrite table with a subset, as a test
sandbox_tab_name=db_table +'_temp' #name for new table in sandbox portion of DB
if columns_match==True:
    df.to_sql(name=sandbox_tab_name, con=engine_sand, index = False, if_exists='replace')
else: print("ERROR: columns in uploaded table do not match those in DB")

Change formatting specifics inside the database

In [19]:
#Reformat columns in sandbox to match format of those in existing table in spatial_database
for xx in range(0,types.shape[0]):
    col_name=types.attname[xx]
    col_type=types.type[xx]
    query=(r"""ALTER TABLE %s
     ALTER COLUMN %s TYPE %s
     USING %s::%s""")%(sandbox_tab_name, col_name, col_type, col_name, col_type)
    engine_sand.execute(query)
    print("done with " +col_name)
print("DONE with" + sandbox_tab_name)

done with station_name
done with extension
done with datetime
done with gid
DONE withdraw_wire_temp


In [20]:
#Set primary key column
engine_sand.execute("ALTER TABLE %s ADD PRIMARY KEY (%s);"%(sandbox_tab_name, pkey))

In [21]:
#Set owner to Administrator
engine_sand.execute("ALTER TABLE %s OWNER TO administrator"%(sandbox_tab_name))

In [23]:
#Set permissions on tables. 
###These are currently not working, as passed thru SQLAlchemy engine; functional when passed direcly in SQL window in PgAdmin.
engine_sand.execute("GRANT SELECT ON TABLE %s TO reader;"%(sandbox_tab_name))
engine_sand.execute("GRANT ALL ON TABLE %s TO administrator"%(sandbox_tab_name))
engine_sand.execute("ALTER TABLE %s OWNER TO administrator;"%(sandbox_tab_name))

### Done! You have uploaded a table to the sandbox which matches structure of the original table (columns and types)